# Task Description

In [1]:
# The task would be to test a long-only stock-based carry/value strategy.

# Carry strategy in stocks would be to long high dividend yield stocks and short the reverse (in this case no need to do short).
# Whole value strategy you should be familiar with.

# Carry trade in rate hike periods work pretty well (for fx especially). We are interested in whether the same hold for value. 
# The underlying assets would be S&P 500 stocks.

# Please take into account that some companies do stock repurchase instead of cash dividend.
# You could try to find adjusted dividend yield data. Time horizon is up to you, but keep in mind potential survivorship bias.
# It might be interesting to check the strategy performance during different periods. 
# Remember to make the backtest a way that they could be implemented (not longing 100 stocks at the same time). 
# Have a nice night and good luck with your presentation on Monday.

# Setup

## Load Packages

In [2]:
#### Import required Packages ####
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import scipy as sp
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error # to calculate the MSE
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score

from statsmodels.graphics.tsaplots import plot_acf # To produce ACF plots
from statsmodels.graphics.tsaplots import plot_pacf # To produce PACF plots
from statsmodels.tsa.seasonal import seasonal_decompose # To decompose Seasons
from statsmodels.tsa.stattools import adfuller, kpss # Tests for Stationarity
from statsmodels.tsa.ar_model import AutoReg # To produce AR models
from statsmodels.stats.anova import anova_lm # To use ANOVA (compare nested models)
from statsmodels.tsa.arima.model import ARIMA # To build ARMA & ARIMA Models
import statsmodels.stats.diagnostic as dg # To get Breusch-Godfrey Test
from statsmodels.stats.stattools import durbin_watson



from datetime import datetime # to transform variables into datetime objects
import math # simple math functions
from math import sqrt # square root function
import statistics # descriptive statistics library
import scipy.stats as stats # descriptive statistics library from scipy
import matplotlib.dates as mdates # date formatting
from matplotlib.collections import PolyCollection, LineCollection # better plot options

## Useful functions for Matrix calculations

In [3]:
def f(x, y): 
    return x + y                             
vecsum = np.vectorize(f)     # Use pd.DataFrame(vecsum(A,B)) to get df of elementwise sum

In [4]:
def g(x, y): 
    return x - y                             
vecdif = np.vectorize(g)     # Use pd.DataFrame(vecdif(A,B)) to get df of elementwise differences

In [5]:
def h(x, y): 
    return x * y                             
vecmult = np.vectorize(h)    # Use pd.DataFrame(vecmult(A,B)) to get df of elementwise multiplication

In [6]:
def i(x, y): 
    return x / y                             
vecdiv = np.vectorize(i)    # Use pd.DataFrame(vecdiv(A,B)) to get df of elementwise division

### Illustrative Example extra für de Herr Vogel

In [7]:
# Chasch de shit au lösche wenns checkt hesch aber mir hets na easy gholfe
A = [[1, 2, 3], 
    [3, 5, 6],
    [7, 8, 9]]
A = pd.DataFrame(A)
A.columns = ["ret1", "ret2", "ret3"]

B = [[0.2, 0.3, 0.5], 
    [0.1, 0.6, 0.3],
    [0.4, 0.2, 0.4]]
B = pd.DataFrame(B)    
B.columns = ["w1", "w2", "w3"]


In [8]:
A

,ret1,ret2,ret3
0,1,2,3
1,3,5,6
2,7,8,9


In [9]:
B

,w1,w2,w3
0,0.2,0.3,0.5
1,0.1,0.6,0.3
2,0.4,0.2,0.4


In [10]:
pd.DataFrame(vecsum(A,B))

,0,1,2
0,1.2,2.3,3.5
1,3.1,5.6,6.3
2,7.4,8.2,9.4


In [11]:
pd.DataFrame(vecdif(A,B))

,0,1,2
0,0.8,1.7,2.5
1,2.9,4.4,5.7
2,6.6,7.8,8.6


In [12]:
pd.DataFrame(vecmult(A,B))

,0,1,2
0,0.2,0.6,1.5
1,0.3,3.0,1.8
2,2.8,1.6,3.6


In [13]:
pd.DataFrame(vecdiv(A,B))

,0,1,2
0,5.0,6.666667,6.0
1,30.0,8.333333,20.0
2,17.5,40.000000,22.5


## Plotstyle

In [14]:
# Seaborn plot style ticks to have nicer looking plots
sb.set_style("ticks")
sb.mpl.rc("figure", figsize=(16,8))
sb.mpl.rc("font", size=14)
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

## LoadData

In [15]:
# Loading Data

data_px = pd.read_excel("02_Data_clean/SPX_px_clean.xlsx", parse_dates=["Date"])
data_px_yly = pd.read_excel("02_Data_clean/SPX_px_clean_yearly.xlsx", parse_dates=["Date"])
data_bv = pd.read_excel("02_Data_clean/SPX_value_clean.xlsx", parse_dates=["Date"])

date_fix = pd.read_excel("02_Data_clean/SPX_px_clean.xlsx")

# Check
print(type(data_bv))
print(data_bv.dtypes)

<class 'pandas.core.frame.DataFrame'>
Date                  datetime64[ns]
AEE UN Equity                float64
ITT UN Equity                float64
UN UN Equity                 float64
APD UN Equity                float64
                           ...      
NTAP UQ Equity               float64
CTXS UQ Equity               float64
9990253D UN Equity           float64
0948669D UN Equity           float64
CMCSK UQ Equity              float64
Length: 501, dtype: object


## Data Cleaning

In [16]:
# Resample monthly returns
data_px.set_index("Date", inplace=True) # set datetimeindex
data_bv.set_index("Date", inplace=True) # set datetimeindex
data_px_yly.set_index("Date", inplace=True) # set datetimeindex
semian_ret = data_px.pct_change(fill_method = None).resample("2BQ").agg(lambda x: ((1+x).prod()-1)) # pct_change creates ordinary returns, resample Monthly and aggregating with the (1+x) -1 formula to get monthly ordinary returns
# df = pd.concat([data, data2], axis=1) # concating dfs


In [17]:
semian_ret = semian_ret.iloc[1: , :]


date_fix['Date'] = date_fix['Date'].replace(['1999-12-31'], '1999-06-30')
date_fix.set_index("Date", inplace=True) # set datetimeindex

midyear_ret = date_fix.pct_change(fill_method = None).resample("4BQ").agg(lambda x: ((1+x).prod()-1)) # pct_change creates ordinary returns

In [18]:
midyear_ret = midyear_ret.iloc[1:-1, :]
midyear_ret

,AEE UN Equity,ITT UN Equity,UN UN Equity,APD UN Equity,RDPL UN Equity,GOLD UN Equity,VZ UN Equity,CAT UN Equity,CVX UN Equity,KO UN Equity,...,PTC UQ Equity,QCOM UQ Equity,SPLS UQ Equity,XLNX UQ Equity,1519128D UQ Equity,NTAP UQ Equity,CTXS UQ Equity,9990253D UN Equity,0948669D UN Equity,CMCSK UQ Equity
Date,,,,,,,,,,,,,,,,,,,,,
2000-06-30,0.030534,-0.091601,-0.210104,-0.081937,0.018614,0.028269,-0.174619,-0.280214,-0.020922,-0.013947,...,-0.593534,-0.659333,-0.259034,0.815806,-0.721477,0.938297,-0.692074,0.223270,-0.195767,-0.199010
2001-06-29,0.265185,0.456809,0.385354,0.484787,-0.053482,-0.167010,0.052892,0.477491,0.067058,-0.216541,...,0.271818,-0.025333,0.040000,-0.500500,0.348436,-0.829814,0.842908,0.275064,-0.421053,0.071604
2002-06-28,0.007260,0.595472,0.087794,0.103168,-0.051484,0.253465,-0.249534,-0.021978,-0.022099,0.244444,...,-0.754825,-0.529925,0.232017,-0.456111,-0.566120,-0.091971,-0.826934,-0.181290,0.514545,-0.450692
2003-06-30,0.025110,-0.085974,-0.166514,-0.186249,-0.153971,-0.073196,-0.012204,0.140552,-0.182147,-0.169464,...,-0.113703,0.307385,-0.059894,0.133749,-0.049425,0.289389,2.428788,-0.109535,0.186074,0.224837
2004-06-30,-0.006577,0.295979,0.257175,0.266130,0.118050,0.165909,-0.098082,0.399785,0.290274,0.077188,...,0.578947,0.948804,0.598808,0.285096,0.223574,0.343516,-0.040555,0.377876,0.113360,-0.050342
2005-06-30,0.244292,0.163939,-0.042857,0.144616,0.209832,0.214425,-0.036345,0.239411,0.214049,-0.157685,...,0.322917,-0.041119,0.092705,-0.200428,-0.028320,0.347100,0.117259,-0.156712,0.205000,0.090514
2006-06-30,-0.073394,0.017053,0.040931,0.073924,-0.033043,0.187801,-0.028430,0.537890,0.094726,0.019431,...,-0.199370,0.193270,0.127492,-0.133180,-0.023331,0.215983,0.808107,0.584158,-0.207469,0.083993
2007-06-29,-0.029505,0.379392,0.375610,0.257353,0.000000,-0.017905,0.275782,0.051289,0.357396,0.215946,...,0.700236,0.082855,-0.024260,0.181898,0.770159,-0.172805,-0.161183,0.092548,0.348406,0.279441
2008-06-30,-0.156091,-0.091678,-0.099613,0.227075,0.000000,0.547988,-0.163623,-0.058110,0.160969,-0.008985,...,-0.206386,0.052086,0.009692,-0.056033,-0.197301,-0.242808,-0.114941,-0.163916,-0.557360,-0.340844


### Create Book to Market Dataframe

In [19]:
# Divide BVPS yearly by SP yearly to get Book to market signal yearly
data_btm = pd.DataFrame(vecdiv(data_bv, data_px_yly))

In [20]:
# Set Datetimeindex again
data_btm.set_index(data_px_yly.index, inplace=True) # set datetimeindex

# Strategy

## Calculate Desired Weights

In [21]:
# The approach is as follows: 
# 1. Take Book to market value as BVPS(t-6mt) / SP(t-6mt)
# 2. for each month calculate the 10 stocks considered for the long strategy
# 3. calculate the PF return
# 4. take into account the transaction costs

In [22]:

cols = (data_btm.columns[0:])
rows = (data_btm.iloc[:,0])

des_weight = pd.DataFrame(index = rows, columns = (cols))
des_weight.set_index(data_px_yly.index, inplace=True) # set datetimeindex
des_weight

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
for i in data_btm.index:
    x = data_btm.loc[i,:].nlargest(10)
    for j in data_btm.columns:
        if j in x.index:
            des_weight.loc[i,j] = 0.1
        else:
            des_weight.loc[i,j] = 0
des_weight = des_weight.set_index(midyear_ret.index)

## Weighted Returns on 30. June

In [60]:
weighted_ret = pd.DataFrame(vecmult(des_weight, midyear_ret))

In [61]:
weighted_ret.set_index(midyear_ret.index, inplace=True) # set datetimeindex

## PF returns weighted per year

In [62]:
ones500x1 = np.ones(500)
pd.DataFrame(ones500x1)

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
495,1.0
496,1.0
497,1.0
498,1.0


In [63]:
pf_ret_unadj = np.matmul(weighted_ret,ones500x1)
pf_ret_unadj = pd.DataFrame(pf_ret_unadj)
pf_ret_unadj.columns =["PF_returns_unadjusted"]
pf_ret_unadj

,PF_returns_unadjusted
Date,
2000-06-30,-0.129123
2001-06-29,0.340499
2002-06-28,-0.275046
2003-06-30,0.015931
2004-06-30,0.384665
2005-06-30,0.049902
2006-06-30,0.021248
2007-06-29,0.164872
2008-06-30,-0.209292


## Actual weights

In [64]:
# 1+return -> for stock returns and for the weighted PF returns -> used for actual weights calculation
midyear_ret_gross = midyear_ret.transform(lambda x: x + 1)
pf_ret_unadj_gross = pf_ret_unadj.transform(lambda x: x + 1)

In [65]:
# Create (23x500) matrix to match midyear return matrix for scaling factor to calculate adjusted weights
ones_matrix = np.ones((23,500))
pf_ret_unadj_gross_matrix = pd.DataFrame(vecmult(pf_ret_unadj_gross, ones_matrix))

In [66]:
# Calculate weight scaling factor by deviding the returns of the assets, per year by the weighted pf return per year
weight_scaleFactor = pd.DataFrame(vecdiv(midyear_ret_gross,pf_ret_unadj_gross_matrix), columns = midyear_ret_gross.columns, index = midyear_ret_gross.index)

In [67]:
des_weight

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
Date,,,,,,,,,,,,,,,,,,,,,
2000-06-30,0,0.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2001-06-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2002-06-28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2003-06-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.1
2004-06-30,0,0.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2005-06-30,0,0.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006-06-30,0,0.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007-06-29,0,0.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-06-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
# Shift by -1 to create w(t-1) but last row becomes N/A
des_weight_lag1 = des_weight.shift(1)

In [71]:
# Calculate actual weights before rebalancing
actual_weight = pd.DataFrame(vecmult(des_weight_lag1, weight_scaleFactor), columns = weight_scaleFactor.columns, index = weight_scaleFactor.index)
actual_weight

,AEE UN Equity,ITT UN Equity,UN UN Equity,APD UN Equity,RDPL UN Equity,GOLD UN Equity,VZ UN Equity,CAT UN Equity,CVX UN Equity,KO UN Equity,...,PTC UQ Equity,QCOM UQ Equity,SPLS UQ Equity,XLNX UQ Equity,1519128D UQ Equity,NTAP UQ Equity,CTXS UQ Equity,9990253D UN Equity,0948669D UN Equity,CMCSK UQ Equity
Date,,,,,,,,,,,,,,,,,,,,,
2000-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-06-29,0.0,0.108677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2002-06-28,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2003-06-30,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2004-06-30,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.068584
2005-06-30,0.0,0.110862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2006-06-30,0.0,0.099589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2007-06-29,0.0,0.118416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2008-06-30,0.0,0.114875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


## Turnover

In [72]:
des_weight

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
Date,,,,,,,,,,,,,,,,,,,,,
2000-06-30,0,0.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2001-06-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2002-06-28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2003-06-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.1
2004-06-30,0,0.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2005-06-30,0,0.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006-06-30,0,0.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007-06-29,0,0.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-06-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
actual_weight

,AEE UN Equity,ITT UN Equity,UN UN Equity,APD UN Equity,RDPL UN Equity,GOLD UN Equity,VZ UN Equity,CAT UN Equity,CVX UN Equity,KO UN Equity,...,PTC UQ Equity,QCOM UQ Equity,SPLS UQ Equity,XLNX UQ Equity,1519128D UQ Equity,NTAP UQ Equity,CTXS UQ Equity,9990253D UN Equity,0948669D UN Equity,CMCSK UQ Equity
Date,,,,,,,,,,,,,,,,,,,,,
2000-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-06-29,0.0,0.108677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2002-06-28,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2003-06-30,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2004-06-30,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.068584
2005-06-30,0.0,0.110862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2006-06-30,0.0,0.099589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2007-06-29,0.0,0.118416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2008-06-30,0.0,0.114875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [88]:
turnover = pd.DataFrame(abs(vecdif(des_weight, actual_weight)), index = weight_scaleFactor.index)
turnover.loc["2000-06-30",:] = des_weight.loc["2000-06-30",:] # Adjust for buying the whole portfolio at the 30.06.2000
turnover

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
Date,,,,,,,,,,,,,,,,,,,,,
2000-06-30,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2001-06-29,0.0,0.108677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2002-06-28,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2003-06-30,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.100000
2004-06-30,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.068584
2005-06-30,0.0,0.010862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2006-06-30,0.0,0.000411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2007-06-29,0.0,0.018416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2008-06-30,0.0,0.114875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [92]:
turnover_yly = np.matmul(turnover,ones500x1)
turnover_yly = pd.DataFrame(turnover_yly, columns = ["Yearly Turnover of PF"])
turnover_yly

,Yearly Turnover of PF
Date,
2000-06-30,1.000000
2001-06-29,1.782873
2002-06-28,1.499348
2003-06-30,0.961053
2004-06-30,0.662733
2005-06-30,0.965762
2006-06-30,0.732010
2007-06-29,0.675869
2008-06-30,1.183595


# Adjust Portfolio Return for Transaction cost and turnover

In [114]:
# Calculate transaction costs yearly by multiplying turnover and transaction costs
c = np.ones(len(turnover_yly)) * 0.002 # 0.2% from Ziegler Slides -> transaction cost corresponding to turnover in %
c = pd.DataFrame(c)
tc_yly = pd.DataFrame(vecmult(turnover_yly,c), index=turnover_yly.index, columns= ["TC_yly"])
tc_yly

,TC_yly
Date,
2000-06-30,0.002000
2001-06-29,0.003566
2002-06-28,0.002999
2003-06-30,0.001922
2004-06-30,0.001325
2005-06-30,0.001932
2006-06-30,0.001464
2007-06-29,0.001352
2008-06-30,0.002367


In [115]:
pf_ret_adj = pd.DataFrame(vecdif(pf_ret_unadj, tc_yly), index = tc_yly.index, columns = ["PF_ret_adj"])
pf_ret_adj

,PF_ret_adj
Date,
2000-06-30,-0.131123
2001-06-29,0.336933
2002-06-28,-0.278044
2003-06-30,0.014009
2004-06-30,0.383340
2005-06-30,0.047970
2006-06-30,0.019784
2007-06-29,0.163521
2008-06-30,-0.211659


In [37]:
# Calculate Turnover by actual weight - desired weight, except first column as this is just